### Import Libraries

In [67]:
PROJECT_ID = "formidable-feat-466408-r6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://agentic_ai_ebooks_bucket"  # @param {type:"string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [53]:
from google import adk
from google.adk.agents import Agent
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part
from google.adk.sessions import VertexAiSessionService
from google.adk.tools import google_search

In [54]:
from vertexai import rag
from google.adk.tools.retrieval.vertex_ai_rag_retrieval import VertexAiRagRetrieval
from vertexai.generative_models import GenerativeModel, Tool
from vertexai import generative_models

In [55]:
import logging
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import BaseModel, Field

In [56]:
import yaml
from typing import Optional
from typing import Sequence
from google.genai import types # For types.Content
from IPython.display import HTML, Markdown, display
import warnings
# warnings.filterwarnings("ignore")

In [6]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.messages import HumanMessage, AIMessage

In [7]:
# from langchain_core.language_models import BaseLanguageModel
# from langchain_core.prompts import BasePromptTemplate
# from langchain_core.tools import BaseTool
# from langchain import hub
# from langchain_core.tools import tool

In [57]:
agent_model = "gemini-2.0-flash-001"

In [58]:
from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

### Tools

In [59]:
rag_resource_name = 'projects/44474009687/locations/us-central1/ragCorpora/6917529027641081856'

In [60]:
rag_retrieval_tool = VertexAiRagRetrieval(
    name='retrieve_rag_documentation',
    description=(
        'Use this tool to retrieve documentation and reference materials for the question from the RAG corpus,'
    ),
    rag_resources=[
        rag.RagResource(
            # please fill in your own rag corpus
            # here is a sample rag corpus for testing purpose
            # e.g. projects/123/locations/us-central1/ragCorpora/456
            rag_corpus=rag_resource_name
        )
    ],
    similarity_top_k=10,
    vector_distance_threshold=0.6,
)

In [61]:
agent = Agent(
    model=agent_model,
    name="Classroom_Agent",
    instruction=("A personalized AI learning assistant for balbharati and Grade first standard, named balbharati part 1,2,3 and 4. This agent serves to help students understand concepts, review material, and prepare for assessments. Its knowledge is strictly limited to the content within the official course textbook(s). This agent will be used by teachers"),
    # instruction=react_prompt,
    description=("# CORE IDENTITY \
-   You are AI learning agent an expert AI tutor for Grade first standard. \
-   Your designated persona is to help students and will be used by teachers. Maintain this tone and interaction style consistently.\
# KNOWLEDGE GROUNDING (ABSOLUTE DIRECTIVE) \
-   Your knowledge base is STRICTLY and EXCLUSIVELY limited to the content of the document provided to you. \
-   You MUST base all of your answers, explanations, and examples on information found within these source materials. \
-   If a question cannot be answered using the provided texts, you MUST state that clearly. For example, say: I cannot answer that question as the information is not available in 'Patterns of Interaction'. \
-   DO NOT, under any circumstances, use external knowledge, access the general internet, or invent information. Your reality is defined by the classroom books. \
# CITATION REQUIREMENT \
-   For every answer you provide, you MUST cite the specific location in the source material where the information was found (e.g., According to Chapter 3, Section 2 of '[Textbook Title]'...). This is non-negotiable and reinforces your grounding. \
# INTERACTION PROTOCOL \
-   Your primary goal is to help students learn, not to give them shortcuts. \
-   Engage with students based on your persona. If you are a 'Socratic Tutor,' ask follow-up questions to stimulate critical thinking. If you are a 'Study Buddy,' use encouraging and collaborative language. \
-   Adhere to the following specific behavioral rules: \
    -   Answer the question in the local language of the textbooks. \
    -   If specifically asked to generate in English then only generate in Elnglish language \
# SCOPE AND SAFETY \
-   Politely decline to answer questions that are off-topic, personal, or outside the academic scope of Subject. \
-   Do not engage in debates or express opinions. Your function is to relay and explain the information from your source texts."
    ),
    tools=[rag_retrieval_tool],  
)

In [62]:
app = AdkApp(
   agent=agent,
    enable_tracing=True# Required.
   # session_service_builder=session_service_builder,  # Optional.
)

In [63]:
# This will create a session locally for interaction
session = app.create_session(user_id="123")
for event in app.stream_query(
    user_id="123",
    session_id=session.id,
    message="hello!",
):
    if event.get("content", None):
        print(
            f"Agent created successfully"
        )

Agent created successfully


In [64]:
session

Session(id='2aec0032-1bdd-4a5f-be0d-3979227ee30e', app_name='default-app-name', user_id='123', state={}, events=[], last_update_time=1753452215.4983802)

In [8]:
# def greetings(query: str):
#   """Tool to greet user."""
#   if 'hello' in query.lower():
#     return {"greeting": "Hello, world"}
#   else:
#     return {"greeting": "Goodbye, world"}

# # Define an ADK agent
# root_agent = adk.Agent(
#     model="gemini-2.0-flash",
#     name='my_agent',
#     instruction="You are an Agent that greet users, always use greetings tool to respond.",
#     tools=[greetings]
# )

In [10]:
# session_service = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)

In [26]:
# runner = adk.Runner(
#     agent=root_agent,
#     app_name='projects/44474009687/locations/us-central1/reasoningEngines/7395007345165598720',
#     session_service=session_service)

In [23]:
# runner = adk.Runner(
#     agent=root_agent,
#     app_name='Test',
#     session_service=session_service)

In [33]:
# Helper method to send query to the runner
# def call_agent(query, session_id, user_id):
#     content = types.Content(role='user', parts=[types.Part(text=query)])
#     events = runner.run(
#       user_id=user_id, session_id=session_id, new_message=content)
#     for event in events:
#         if event.is_final_response():
#             final_response = event.content.parts[0].text
#             print("Agent Response: ", final_response)

In [18]:
# agent.name.split("/")[-1]

'Sahayak_Agent'

In [18]:
# Create a session
# session = await session_service.create_session(
#        app_name='projects/44474009687/locations/us-central1/reasoningEngines/7395007345165598720',
#        user_id='memory_test_5')

In [65]:
for event in app.stream_query(
    user_id="123",
    session_id=session.id,
    message="What are the 21st century skills",
):
    print(event)

{'content': {'parts': [{'text': "२१ व्या शतकातील कौशल्ये खालीलप्रमाणे आहेत:\n\n*   चिकित्सक विचार (Critical Thinking)\n*   माहिती (Information)\n*   लवचीकता (Flexibility)\n*   नेतृत्व (Leadership)\n*   पुढाकार (Initiative)\n*   उत्पादकता (Productivity)\n*   सामाजिक कौशल्ये (Social Skills)\n*   माध्यम (Media)\n*   तंत्रज्ञान (Technology)\n*   सर्जनशीलता (Creativity)\n*   सहकार्य (Collaboration)\n*   संप्रेषण (Communication)\n*   अध्ययन कौशल्ये (Learning Skills)\n*   साक्षरता कौशल्ये (Literacy Skills)\n*   जीवन कौशल्ये (Life Skills) ('Balbharati Part 1', 'Balbharati Part 2', 'Balbharati Part 3' आणि 'Balbharati Part 4')"}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'retrieved_context': {'rag_chunk': {'page_span': {'first_page': 12, 'last_page': 13}, 'text': 'लिही.\r\n(Write.)\r\nऐक व म्हण.\r\n(Listen and say.)\r\nकृती कर.\r\n(Do activity.)\r\nमैदानी खेळ\r\n(Outdoor games)\r\nरंग भर. \r\n(Fill the colours.)\r\nचर्चा कर.\r\n(Discuss.)\r\nकोडी सोडव.\r\n(Solve the puzzles

In [66]:
display_name = "Classroom_Agent"
description = "An agent that has access to the classroom RAG engine"

### Agent Deployment

In [70]:
remote_agent = agent_engines.create(
    app,
    requirements=[
        "google-adk (>=0.0.2)",
        "google-cloud-aiplatform[agent_engines] (>=1.91.0,!=1.92.0)",
        "google-genai (>=1.5.0,<2.0.0)",
        "llama-index",
        "pydantic (>=2.10.6,<3.0.0)"
    ],
    display_name=display_name,
    description=description
)

Identified the following requirements: {'google-cloud-aiplatform': '1.103.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-adk (>=0.0.2)', 'google-cloud-aiplatform[agent_engines] (>=1.91.0,!=1.92.0)', 'google-genai (>=1.5.0,<2.0.0)', 'llama-index', 'pydantic (>=2.10.6,<3.0.0)', 'cloudpickle==3.1.1']
Using bucket agentic_ai_ebooks_bucket
Wrote to gs://agentic_ai_ebooks_bucket/agent_engine/agent_engine.pkl
Writing to gs://agentic_ai_ebooks_bucket/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agentic_ai_ebooks_bucket/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/44474009687/locations/us-central1/reasoningEngines/8877817522477334528/operations/1214682962524635136
View progress and logs at https://console.cloud.google.com/logs/query?projec

In [ ]:
# resource_name='projects/925341040009/locations/us-central1/reasoningEngines/719599574053814272'

# remote_agent = agent_engines.update(
#     agent_engine=agent,
#     resource_name=resource_name,
#     display_name=display_name,
#     description=description
# )

In [74]:
for event in remote_agent.stream_query(
    user_id="1",
    # session_id=SESSION_ID, # Optional. you can pass in the session_id when querying the agent
    message="hello",
):
    print(1)
    print(event)

In [ ]:
# remote_agent.delete()